In [1]:
# Import necessary modules
import spacy, numpy as np
!python -m spacy download en_vectors_web_lg
from keras.models import model_from_json, Model
from keras.layers import Dense, Dropout, LSTM, Input, Reshape, concatenate, Activation
from keras.applications.vgg19 import preprocess_input
from keras.applications import VGG19
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import plot_model
labels = np.load('data/encoded_labels.npy')

Using TensorFlow backend.


In [0]:
def get_img_vec(img_path):
    vgg_base_model = VGG19(weights='imagenet')
    vgg19_model = Model(vgg_base_model.input, vgg_base_model.layers[-2].output)
    img = load_img(img_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    img_vec = np.zeros((1, 4096))
    img_vec[0,:] = vgg19_model.predict(img)[0]
    return img_vec

In [0]:
def get_ques_vec(ques):
    word_embeds = spacy.load('en_vectors_web_lg')
    tokens = word_embeds(ques)
    ques_vec = np.zeros((1, 30, 300))
    for j in range(len(tokens)):
            ques_vec[0,j,:] = tokens[j].vector
    return ques_vec

In [0]:
vqa_model = model_from_json(open('arch/vqa_model_arch.json').read())
plot_model(vqa_model, 'arch/vqa_model_arch.png')
vqa_model.load_weights('data/vqa_model_weights.hdf5')

In [0]:
def predictions(img_path, ques):
    ques_vector = get_ques_vec(ques)
    img_vector = get_img_vec(img_path)
    preds = vqa_model.predict([ques_vector, img_vector])[0]
    top = np.argsort(preds)[-10:][::-1]
    print('I am\033[1m', str(round(preds[top[0]]*100, 2)) + '%\033[0m', 'sure that the answer is \033[1m' + labels[top[0]] + '\033[0m, but I also think: ')
    for ind in top[1:]:
        print(str(round(preds[ind]*100, 2)) + '%', labels[ind])

    return load_img(img_path, target_size=(250, 400))

In [0]:
predictions('test.jpg', 'what color is the helmet')

I am 48.13% sure that the answer is blue, but I also think: 
32.83% white
7.19% black
4.36% silver
3.95% red
0.83% green
0.66% gray
0.51% red and white
0.46% purple
0.31% blue and white
